In [2]:
import numpy as np
import pandas as pd

train = pd.read_csv('train.csv',parse_dates=["Date"])
test = pd.read_csv('test.csv',parse_dates=["Date"])

In [3]:
# Rename variables
train.rename(columns = {'Country_Region':'Country','Province_State':'State'},inplace=True)
test.rename(columns = {'Country_Region':'Country','Province_State':'State'},inplace=True)

# Encoding country feature
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
train['Country'] = labelencoder.fit_transform(train['Country'])
test['Country'] = labelencoder.fit_transform(test['Country'])

# Extracting features from Date feature
train['Dayofyear'] = train['Date'].dt.dayofyear
train['Dayofweek'] = train['Date'].dt.dayofweek
train['Weekofyear'] = train['Date'].dt.weekofyear
train['Month'] = train['Date'].dt.month
train['Dayofmonth'] = train['Date'].dt.day

test['Dayofyear'] = test['Date'].dt.dayofyear
test['Dayofweek'] = test['Date'].dt.dayofweek
test['Weekofyear'] = test['Date'].dt.weekofyear
test['Month'] = test['Date'].dt.month
test['Dayofmonth'] = test['Date'].dt.day

train1 = train[train['State'].isna()==True][:]
test1 = test[test['State'].isna()==True][:]

train1.drop('State',axis=1, inplace=True)
test1.drop('State',axis=1, inplace=True)

x = train1[["Country", "Dayofyear", "Dayofweek","Weekofyear","Month","Dayofmonth"]]
y = train1[["ConfirmedCases", "Fatalities"]]
from sklearn.tree import DecisionTreeRegressor
regressor1_1 = DecisionTreeRegressor(random_state = 0)
regressor1_1.fit(x ,y["ConfirmedCases"])
x = x.assign(ConfirmedCaseshat = np.array(regressor1_1.predict(x))) 
from sklearn.tree import DecisionTreeClassifier
regressor1_2 = DecisionTreeClassifier(random_state = 0)
regressor1_2.fit(x ,y["Fatalities"])
test_x_1 = test1[["Country", "Dayofyear", "Dayofweek","Weekofyear","Month","Dayofmonth"]]
cases_predict = regressor1_1.predict(test_x_1)
test_x_1 = test_x_1.assign(ConfirmedCaseshat = np.array(cases_predict))
fat_predict = regressor1_2.predict(test_x_1)
ConfirmedCases = pd.DataFrame(cases_predict.astype(int))
Fatalities = pd.DataFrame(fat_predict.astype(int))
test1 = test1.assign(ConfirmedCases = np.array(ConfirmedCases))
test1 = test1.assign(Fatalities = np.array(Fatalities))

In [215]:
train2 = train[train['State'].isna()==False][:]
test2 = test[test['State'].isna()==False][:]
from sklearn.preprocessing import LabelEncoder
labelencoder1 = LabelEncoder()
train2['State'] = labelencoder1.fit_transform(train2['State'])
test2['State'] = labelencoder1.fit_transform(test2['State'])
x = train2[["State","Country", "Dayofyear", "Dayofweek","Weekofyear","Month","Dayofmonth"]]
y = train2[["ConfirmedCases", "Fatalities"]]
from sklearn.tree import DecisionTreeRegressor
regressor2_1 = DecisionTreeRegressor(random_state=0)
regressor2_1.fit(x ,y["ConfirmedCases"])
x = x.assign(ConfirmedCaseshat = np.array(regressor2_1.predict(x))) 
from sklearn.tree import DecisionTreeRegressor
regressor2_2 = DecisionTreeRegressor(random_state=0)
regressor2_2.fit(x ,y["Fatalities"])
test_x_2 = test2[["State","Country", "Dayofyear", "Dayofweek","Weekofyear","Month","Dayofmonth"]]
cases_predict = regressor2_1.predict(test_x_2)
test_x_2 = test_x_2.assign(ConfirmedCaseshat = np.array(cases_predict))
fat_predict = regressor2_2.predict(test_x_2)
ConfirmedCases = pd.DataFrame(cases_predict.astype(int))
Fatalities = pd.DataFrame(fat_predict.astype(int))
test2 = test2.assign(ConfirmedCases = np.array(ConfirmedCases))
test2 = test2.assign(Fatalities = np.array(Fatalities))

In [216]:
result = test1.append(test2,sort='ForecastId')
result.sort_values(by=['ForecastId'],inplace=True)
submission = result[['ForecastId','ConfirmedCases','Fatalities']]

In [217]:
submission.to_csv('submission.csv',index=False)

In [219]:
from vecstack import stacking

ModuleNotFoundError: No module named 'vecstack'